# Computational concrete compositions in HTML with absolute positioning

By [Allison Parrish](https://www.decontextualize.com/)

The goal of this notebook is to show you how to write Python code that arranges units of language on a page. Along the way, I'm going to reference some well-known works of concrete poetry (both non-digital and digital) as examples.

Specifically, we're going to write Python code that *generates HTML and CSS*.

There are lots of ways to do computational layout. Almost all of these methods involve writing a computer program that either produces markup language, or manipulates an intermediate representation of a markup language (like [DOM](https://en.wikipedia.org/wiki/Document_Object_Model)). We could use [p5.js](https://p5js.org/), [Paper.js](http://paperjs.org/) or [Rune.js](http://runemadsen.github.io/rune.js/) to work in the browser with JavaScript, or [Basil.js](http://basiljs.ch/) to script Adobe InDesign. We could write Python programs that produce [PostScript](https://en.wikipedia.org/wiki/PostScript) or [LaTeX](https://www.latex-project.org/) or even [Gcode](https://en.wikipedia.org/wiki/G-code) directly, or that script third-party layout tools like [InkScape](https://inkscape.org/develop/extensions/). Later notebooks in this series will explore a computational design library for Python called [Flat](https://xxyxyz.org/flat) that (among other things) exports [SVG](https://en.wikipedia.org/wiki/Scalable_Vector_Graphics). We'll be using this primarily for the purpose of drawing letter-like shapes and parsing font shape data.

## Why HTML/CSS?

Generating HTML/CSS turns out to be a good place to start with computational layout! Nearly every mobile phone and personal computer has a powerful HTML interpreter installed (i.e., a web browser), so no additional proprietary software is necessary. Both HTML and CSS remain mainstays of programming education, and even if you're not already familiar with them, it's easy to pick up the basics. Recent versions of CSS have made it possible to do very sophisticated layouts in HTML—even things that you would have previously been forced to implement in a vector graphics tool. HTML and CSS as tools remain primarily focused on layouts for screens, but support for the printed page is surprisingly robust. (See the "Further Resources" section below for links to HTML/CSS tutorials and more information on using HTML/CSS to produce printed artifacts.)

But maybe the best reason to use HTML/CSS for computational layout is that browsers are already pretty good at supporting advanced font features and [complex text shaping](https://en.wikipedia.org/wiki/Complex_text_layout) (including ligatures and bidirectional text) required for many of the world's languages.

## Why computation?

Concrete compositions are conventionally done "by hand," and there's nothing in this notebook you couldn't have written in HTML by hand in a text editor with a little patience. I'm interested in using computation to do these kinds of compositions for a few reasons. The first is that computation makes it easy to quickly iterate on variations of even very simple ideas, especially when we're using complex mathematical formulas and algorithms that are difficult to calculate by hand (like trigonometric functions or random number generators). Computation also allows us to work at scale: the effort needed to lay out *n* words by hand scales linearly with *n*, but a computer can apply the same procedure over and over to an arbitrary number of words with little additional human effort. Both of these factors contribute to a sense that computation facilitates certain ways of working with text and layout (and making other ways of working with text more difficult). Because computation and digital media are so central to our lives, it's worth exploring what kinds of layout computation facilitates.

## Preliminaries

We'll need Python's `random` and `math` libraries, because we're going to do some random stuff with math:

In [1]:
import random
import math

The following function displays HTML source code in Jupyter Notebook as an embedded [iframe](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/iframe). The `isolated=True` argument ensures that any styles we define in the generated HTML will not leak back into the notebook itself:

In [2]:
from IPython.display import display, HTML
def show_html(src):
    return display(HTML(src), metadata=dict(isolated=True))

Let's test it a little bit. This cell creates a small HTML snippet with a single paragraph tag whose background is set to a good gif:

In [3]:
html_src = """
<body>
<p style="height: 240px; background-image: url(https://i.giphy.com/media/OmK8lulOMQ9XO/giphy-downsized.gif);">Hello!</h1>
</body>
</html>
"""
show_html(html_src)

The following cell defines an HTML template that we'll use in many of the following examples. This template sets a basic style on the HTML tag, making sure that it's at least 32em tall (`em` is a CSS unit that corresponds with the default font size) and that any elements that are positioned outside of this range are hidden. It leaves space for you to [interpolate](interpolating-strings.ipynb) a `title` and some `content`.

In [4]:
html_tmpl = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>{title}</title>
    <style>
    html {{ min-height: 32em; overflow: hidden; }}
    </style>
</head>
<body>
{content}
</body>
</html>"""

So, to interpolate a title and content into this template, we might write:

In [5]:
interp_src = html_tmpl.format(title="My first HTML page", content="<h1>This is a test!</h1>")
print(interp_src)

... which we can then display:

In [6]:
show_html(interp_src)

The following bit of code saves this HTML to a file. You can copy/paste this code anywhere in the notebook; just make sure to change the name of the variable whose content you want to save, and the filename you're saving it to:

In [7]:
open("test.html", "w").write(interp_src)

234

You can then open the file up with any web browser. If your system supports the `open` command, running the following cell will open this file up in a new tab:

In [8]:
!open test.html

## Absolute positioning

The kind of HTML/CSS we're going to be generating in this notebook is a little bit unusual in comparison to most HTML/CSS. Because we're concerned with visual materiality, we're not going to spare a thought for [semantic markup](https://en.wikipedia.org/wiki/Semantic_HTML) (although I think it would be an interesting exercise to try to integrate concrete composition with explicit semantics!). We're also not developing a website that people will have to maintain over time, so our code does not have to be reusable or even especially readable. It's not especially important for our layout to be [responsive](https://en.wikipedia.org/wiki/Responsive_web_design), though many of the examples below are designed to be resizable based on the width of the browser window.

Given all of the above, the easiest way to proceed is to make our HTML/CSS entirely from *absolutely positioned `<div>` elements with inline styles*. The `<div>` element is a block element that has no predetermined semantic meaning; *absolute positioning* is a way to position an element on the screen in a particular location, unaffected by the flow of the document. An inline style is CSS specified in a `style` attribute on the element itself, instead of in an external stylesheet. Here's an example:

In [9]:
my_div = "<div style='position: absolute; left: 5%; top: 20%;'>hello!</div>"
html_src = html_tmpl.format(title="hello", content=my_div)
show_html(html_src)

The `<div>` tag's style sets `position: absolute;` to tell the browser to remove the element from the regular document flow and the `left: 5%; top: 20%;` tells the browser to position this element 5% of the window's width from the left and 10% of the window's width from the top.

CSS supports many other [units of length](https://developer.mozilla.org/en-US/docs/Learn/CSS/Building_blocks/Values_and_units#Numbers_lengths_and_percentages). In this notebook I like using [percentage](https://developer.mozilla.org/en-US/docs/Web/CSS/percentage) (`%`) because it lets us fill up the entire window easily. But you might want to use an absolute length unit like millimeters (`mm`) or points (`pt`, 1/72") if you're designing for print, or like pixels (`px`) if you want pixel-perfect positioning on screen. In some of the examples below, I use `vh`, which is one percent of the height of the [viewport](https://developer.mozilla.org/en-US/docs/Glossary/viewport), as an easy way to have a unit that responds to the size of the window but is also the same horizontally and vertically (which `%` is not guaranteed to be).

The following example is a bit more sophisticated: in a loop, it creates one thousand random `<div>`s and adds them to a list. This list is then joined together and interpolated into the HTML template:

In [10]:
divs = []
for i in range(1000):
    x = random.random() * 100
    y = random.random() * 100
    this_div = f"<div style='position: absolute; left: {x}%; top: {y}%;'>★</div>"
    divs.append(this_div)
html_src = html_tmpl.format(title="Stars", content="".join(divs))

In [11]:
show_html(html_src)

You're not limited to just using the `left` and `top` style attributes! Any CSS style is fair game. In the example below, I assign a random size and transparency to the stars, using the `font-size` CSS style, the `color` CSS style, and the `rgba` CSS [color unit](https://developer.mozilla.org/en-US/docs/Learn/CSS/Building_blocks/Values_and_units#Color):

In [12]:
divs = []
for i in range(1000):
    x = random.random() * 100
    y = random.random() * 100
    size = random.randrange(4, 32)
    alpha = random.random()
    this_div = f"<div style='position: absolute; left: {x}%; top: {y}%; font-size: {size}pt; color: rgba(0,0,0,{alpha})'>★</div>"
    divs.append(this_div)
html_src = html_tmpl.format(title="Stars of different sizes", content="".join(divs))

In [13]:
show_html(html_src)

[Here's a good list of CSS properties to try out](https://htmldog.com/references/css/properties/), broken down by category.

## Div-making, simplified

Since we're mostly working with absolutely-positioned `<div>`s with inline styles, I wrote a function below that wraps the `<div>`-making process. The first parameter is the `<div>`'s content, and the remaining named parameters are interpreted as CSS properties to insert into an inline style. If a `position` attribute isn't already specified, it sets it to `absolute`:

In [14]:
def mkdiv(content, **kwargs):
    if 'position' not in kwargs:
        kwargs['position'] = 'absolute'
    style_str = ' '.join([": ".join((k.replace('_', '-'), v))+";" for k, v in kwargs.items()])
    return f"<div style='{style_str}'>{content}</div>"

Here's the function in action:

In [15]:
mkdiv("hello", top="50%", left="50%", font_family="Helvetica")

"<div style='top: 50%; left: 50%; font-family: Helvetica; position: absolute;'>hello</div>"

Python lets you write keyword arguments on separate lines, so you can format this nicely in your notebook like so:

In [16]:
my_fun_div = mkdiv("nice!",
                   top="50%",
                   left="50%",
                   font_family="Helvetica")
print(my_fun_div)

<div style='top: 50%; left: 50%; font-family: Helvetica; position: absolute;'>nice!</div>


Note that Python keyword arguments can't contain hyphens (`-`). But lots of CSS properties have hyphens! When calling the `mkdiv` function, specify these instead with underscores (`_`), as I did with `font_family` in the example above.

## Positioning around the center

If you're sharp-eyed, you might notice that there are stars overlapping the right and bottom edges of the window in the above example, but not the left and top edges. This is because the `top` and `left` CSS properties set the position of the `<div>` according to its top left corner (i.e., the top left corner of the div will be positioned at the given coordinates, with the rest of the div to the right and below). You can also use `bottom` instead of `top` and `right` instead of `left`, as illustrated in the following example:

In [17]:
my_divs = [
    mkdiv("lights", top="50%", left="50%", font_size="72pt", border="1px black solid"),
    mkdiv("lights", bottom="50%", left="50%", font_size="72pt", border="2px cyan solid"),
    mkdiv("lights", bottom="50%", right="50%", font_size="72pt", border="3px blue solid"),
    mkdiv("lights", top="50%", right="50%", font_size="72pt", border="4px red solid")
]
html_src = html_tmpl.format(title="Positioning", content="".join(my_divs))
show_html(html_src)

The CSS `border` property draws a border around the `<div>`'s bounding box. The above code draws four divs, each with different combinations of `top`, `bottom`, `left` and `center` and different border colors, so you can see how they work.

Unfortunately, there are no `center-x` or `center-y` parameters to position a `<div>` according to its center point! Instead, you can set the `<div>`'s position with `top` and `left` and use the CSS `transform` property to move the element half of its width up and to the left:

In [18]:
this_div = mkdiv("lights",
                 top="50%",
                 left="50%",
                 transform="translate(-50%, -50%)",
                 font_size="72pt",
                 border="1px black solid")
html_src = html_tmpl.format(title="Center a div", content=this_div)
show_html(html_src)

Putting all of this together, we can redo the random stars example, but centering the stars around the randomly chosen coordinate:

In [19]:
divs = []
for i in range(1000):
    x = random.random() * 100
    y = random.random() * 100
    size = random.randrange(4, 32)
    alpha = random.random()
    this_div = mkdiv("★",
                     left=f"{x}%",
                     top=f"{y}%",
                     transform="translate(-50%, -50%)",
                     font_size=f"{size}pt",
                     color=f"rgba(0,0,0,{alpha})")
    divs.append(this_div)
html_src = html_tmpl.format(title="Centered stars of different sizes", content="".join(divs))
show_html(html_src)

> Exercise: Expand the "vocabulary" of this piece by finding other characters that resemble stars and modifying the code such that it selects randomly among these characters when placing stars on the page.

## Exploding texts

We'll cover a few techniques in this notebook for making concrete compositions. A frequently used strategy in concrete poetry is to take a text and break it up into units, then place those units on the page in patterns.

The following example does this very thing, iterating over a string character by character and placing those characters on the screen in random positions:

In [20]:
src = "Mother said there'd be days like these"
divs = []
for ch in src:
    this_div = mkdiv(ch,
                     position="absolute",
                     top=f"{random.randrange(25, 75)}%",
                     left=f"{random.randrange(25, 75)}%",
                     font_size="16pt")
    divs.append(this_div)
html_src = html_tmpl.format(title="exploded characters", content="".join(divs))
show_html(html_src)

And this example does something similar, breaking the text up instead by word:

In [21]:
src = "Mother said there'd be days like these"
divs = []
for word in src.split(): # splitting on white space
    this_div = mkdiv(word,
                     position="absolute",
                     top=f"{random.randrange(25, 75)}%",
                     left=f"{random.randrange(25, 75)}%",
                     font_size="16pt")
    divs.append(this_div)
html_src = html_tmpl.format(title="exploded words", content="".join(divs))
show_html(html_src)

An example of applying this technique but with an external file:

In [22]:
src = open("frost.txt").read()
divs = []
for word in src.split(): # splitting on white space
    this_div = mkdiv(word,
                     top=f"{random.randrange(10, 90)}%",
                     left=f"{random.randrange(10, 90)}%",
                     transform="translate(-50%, -50%)",
                     font_size="16pt")
    divs.append(this_div)
html_src = html_tmpl.format(title="exploded words", content="".join(divs))
show_html(html_src)

> Exercise: Implement a "word cloud" visualization. Use `Counter` from Python's `collections` module ([tutorial here](https://gist.github.com/aparrish/4b096b95bfbd636733b7b9f2636b8cf4)) to count the number of times each word occurs in the text, then place words on the page with a font size proportional to each word's frequency.

## Visual arrangements

Random placement is easy and nice, but hardly the limit of our capabilities. Let's look at a few other techniques that are often used in concrete composition.

### Grids

We'll start with the grid. To make a grid, pick the number of rows and columns, then use the index variables from nested `for` loops to calculate the position at which to place elements. This code makes a grid of random emoji, leaving positions in the grid empty at random:

In [23]:
emoji_src = ['🐱', '😸', '😹', '😺', '😻', '😼', '😽', '😾', '😿', '🙀']
grid_size = 12
divs = []
for i in range(grid_size):
    for j in range(grid_size):
        x_pos = 25 + (50 / grid_size) * i
        y_pos = 25 + (50 / grid_size) * j
        if random.random() < 0.67:
            this_div = mkdiv(random.choice(emoji_src),
                             top=f"{x_pos}%",
                             left=f"{y_pos}%")
        divs.append(this_div)
html_src = html_tmpl.format(title="emoji grid", content="".join(divs))
show_html(html_src)

> Exercise: Add a bit of randomness to the grid coordinates.

This example lays out the words of *The Road Not Taken* in an evenly-spaced two-dimensional grid, using nested `for` loops and a variable to keep track of the current index as it moves through the list of words:

In [24]:
src = open("frost.txt").read()
words = src.split()
grid_size = int(math.sqrt(len(words)))
current_index = 0
divs = []
for i in range(grid_size):
    for j in range(grid_size):
        x_pos = (100 / grid_size) * i
        y_pos = (100 / grid_size) * j
        this_div = mkdiv(words[current_index],
                         top=f"{x_pos}%",
                         left=f"{y_pos}%")
        divs.append(this_div)
        current_index += 1
html_src = html_tmpl.format(title="frost grid", content="".join(divs))
show_html(html_src)

Drawing on the previous two examples, the following code generates concrete poems in the style of [Marc Adrian's "A Semantic infra- and meta-structure"](http://dada.compart-bremen.de/item/artwork/252). It does this by placing each character in the text in a grid, and assigning each character a random font size:

In [25]:
src = open("frost.txt").read()
letters = list(src)
grid_size = int(math.sqrt(len(letters)))
current_index = 0
divs = []
for i in range(grid_size):
    for j in range(grid_size):
        x_pos = 5 + (90 / grid_size) * i
        y_pos = 5 + (90 / (grid_size)) * j
        this_div = mkdiv(letters[current_index],
                         top=f"{x_pos}%",
                         left=f"{y_pos}%",
                         font_family="Helvetica",
                         font_size=f"{random.randrange(4, 48)}",
                         transform="translate(-50%, -50%)")
        divs.append(this_div)
        current_index += 1
html_src = html_tmpl.format(title="frost grid", content="".join(divs))
show_html(html_src)

### Points along lines

Another strategy is to draw letters or words arranged along lines (though still retaining their fixed vertical orientation). To make this happen, we need some way to determine *points along the line* that connects two arbitrary points. The following function does just this. Pass it two coordinates (in the form of tuples like `(4, 15)` or `(5.5, 1.23)` and the number of interpolated points you want, and it will return a list of coordinates joining the two points (including the start and end point that you specified):

In [26]:
import numpy as np
def interp(p1, p2, n=10):
    p1 = np.array(p1)
    p2 = np.array(p2)
    pts = []
    for i in range(n):
        pt = (p2 * (i/(n-1))) + (p1 * (1-(i/(n-1))))
        pts.append(tuple(pt))
    return pts

interp((0, 0), (5, 21), n=5)

[(0.0, 0.0), (1.25, 5.25), (2.5, 10.5), (3.75, 15.75), (5.0, 21.0)]

The example below uses this function to create a list of points along a line that has the same number of points as characters in the source string. Then, using a `for` loop, it iterates over the number of points and creates a `<div>` at each point for each character:

In [27]:
src = "Mother said there'd be days like these"
divs = []
start = (25, 25)
end = (75, 75)
pts = interp(start, end, n=len(src))
for i in range(len(src)):
    ch = src[i]
    pt = pts[i]
    this_div = mkdiv(ch,
                     top=f"{pt[1]}%",
                     left=f"{pt[0]}%",
                     font_size="16pt")
    divs.append(this_div)
html_src = html_tmpl.format(title="characters along lines", content="".join(divs))
show_html(html_src)

By changing the font size along the length of the line, we can create a 3D effect similar to [Carl Fernbach-Flarsheim's "The Boolean Image/Conceptual Typewriter"](https://rhizome.org/editorial/2013/jan/16/prosthetic-knowledge-picks-typewriter/):

In [28]:
words = [
  "machinery",
  "madness",
  "magnificence",
  "mahogany",
  "mailing",
  "mainframe",
  "maintenance",
  "majority",
  "manga",
  "mango",
  "manifesto",
  "mantra",
  "manufacturer",
  "maple",
  "martin",
  "martyrdom"
]
# use the length of the longest word to determine how many points in the line interpolation
longest = max([len(w) for w in words])
divs = []
# note: number of words needs to match grid_size * grid_size
grid_size = 4
current_index = 0
for i in range(grid_size): # grid x
    for j in range(grid_size): # grid y
        # x1, y1 = first letter of word
        # x2, y2 = destination of line (into "distance")
        x1 = 10 + (80 / grid_size) * i
        y1 = 10 + (80 / grid_size) * j
        x2 = 50 + (40 / grid_size) * i
        y2 = 50 + (40 / grid_size) * j
        word = words[current_index]
        pts = interp((x1, y1), (x2, y2), longest)
        current_index += 1
        for k in range(len(word)): # iterating through characters/points
            pt = pts[k]
            this_div = mkdiv(word[k],
                             top=f"{pt[1]}%",
                             left=f"{pt[0]}%",
                             transform="translate(-50%, -50%)",
                             font_size=f"{24 - (k*2)}pt",
                             font_family="Helvetica")
            divs.append(this_div)
html_src = html_tmpl.format(title="boolean image homage", content="".join(divs))
show_html(html_src)

You can also use this to make rudimentary shapes. In the following example, I create a list of string "parts" that has the same number of elements as the points in a polygon (excluding the start point, which is repeated in order to close the shape). In the outer `for` loop, I iterate over the indices of points in the polygon, and use `interp()` to calculate a line between the coordinates for each point and the subsequent point, resulting in a list of interpolated points with the same number of characters as the string "part" that corresponds with the side of the polygon. In the inner `for` loop, I iterate over the indices of the list of points, and make a `<div>` for each character from the current part at that point.

In [29]:
divs = []
parts = ["Mother ", "said there'd ", "be days ", "like these "]
polygon = [(30, 25), (80, 25), (70, 75), (20, 75), (30, 25)]
for i in range(len(polygon) - 1):
    side_text = parts[i]
    pts = interp(polygon[i], polygon[i+1], n=len(side_text))
    for j in range(len(pts)):
        ch = side_text[j]
        pt = pts[j]
        this_div = mkdiv(ch,
                     top=f"{pt[1]}%",
                     left=f"{pt[0]}%",
                     font_size="16pt",
                     font_family="Courier")
        divs.append(this_div)
html_src = html_tmpl.format(title="mother parallelogram", content="".join(divs))
show_html(html_src)

Using a bit of trigonometry, we can even use this function to create a radial effect. (Need a refresher? See [Polar and Cartesian coordinates](https://www.mathsisfun.com/polar-cartesian-coordinates.html).)

In [30]:
words = [
  "machinery",
  "madness",
  "magnificence",
  "mahogany",
  "mailing",
  "mainframe",
  "maintenance",
  "majority",
  "manga",
  "mango",
  "manifesto",
  "mantra",
  "manufacturer",
  "maple",
  "martin",
  "martyrdom"
]
# use the length of the longest word to determine how many points in the line interpolation
longest = max([len(w) for w in words])
divs = []
for i, word in enumerate(words):
    # step around a circle (math.tau = 2*pi = 360 degrees)
    theta = (math.tau / len(words)) * i
    # inner coord
    x1 = 50 + math.cos(theta) * 8
    y1 = 50 + math.sin(theta) * 8
    # outer coord
    x2 = 50 + math.cos(theta) * 40
    y2 = 50 + math.sin(theta) * 40
    pts = interp((x1, y1), (x2, y2), len(word))
    for j, ch in enumerate(word):
        this_div = mkdiv(ch,
                         top=f"{pts[j][1]}vh",
                         left=f"{pts[j][0]}vh",
                         font_family="Courier")
        divs.append(this_div)
html_src = html_tmpl.format(title="radial m poem", content="".join(divs))
show_html(html_src)

## Rotation with CSS

The CSS `transform` property is incredibly powerful! We've already used its `translate` command to set positioning around the center of a `<div>`. The `rotate` command makes it possible to *rotate* an element around its origin. If the `translate` command is set to `translate(-50%, -50%)`, the `rotate` command rotates the `<div>` around its center. Here's an example, which rotates a single phrase around a center point:

In [31]:
divs = []
n = 10
for i in range(n):
    degrees = i * (180 / n) # a half circle
    this_div = mkdiv("mother said there'd be days like these",
                      top="50%",
                      left="50%",
                      transform=f"translate(-50%, -50%) rotate({degrees}deg)")
    divs.append(this_div)
html_src = html_tmpl.format(title="centered", content="".join(divs))
show_html(html_src)

By gradually increasing the `height` attribute of the tag, we can create a spiral effect:

In [32]:
words = open("frost.txt").read().split()
divs = []
angle_step = 15
for i, word in enumerate(words):
    angle = i * angle_step
    this_div = mkdiv(word,
                     top=f"50%",
                     left=f"50%",
                     height=f"{12+i*3}pt",
                     font_size="12pt",
                     #border="1px black solid", # uncomment to see the boxes
                     transform=f"translate(-50%, -50%) rotate({angle}deg)")
    divs.append(this_div)
html_src = html_tmpl.format(title="frost spiral", content="".join(divs))
show_html(html_src)

Keeping the height constant lays out the divs in a circle:

In [33]:
divs = []
n = 14
for i in range(n):
    degrees = i * (360 / n) 
    this_div = mkdiv("C",
                     top="50%",
                     left="50%",
                     width="1em",
                     height="4em",
                     font_size="32px",
                     transform=f"translate(-50%, -50%) rotate({degrees}deg)")
    divs.append(this_div)
html_src = html_tmpl.format(title="centered", content="".join(divs))
show_html(html_src)

Combining all of this together: we can create an homage to Mary Ellen Solt's [Geranium](http://ubu.com/historical/solt/flowers/pdf/Solt_Geranium_1963.pdf). This example uses two different kinds of rotation: first, it uses polar coordinates to determine the center of each "bunch"; then it uses CSS `height` plus `rotate` to position the individual letters in each "bunch" around that center coordinate. Tricky, but the benefit is that you can change the text of the poem to anything you like:

In [34]:
divs = []
n = 14
src = "ALPHABET"
for i in range(len(src)):
    ch = src[i]
    center_x = 50 + (30 * math.cos(i * (math.tau / len(src))))
    center_y = 50 + (30 * math.sin(i * (math.tau / len(src))))
    for j in range(n):
        degrees = j * (360 / n)
        this_div = mkdiv(src[i],
                          position="absolute",
                          top=f"{center_y}vh",
                          left=f"{center_x}vh",
                          width="1em",
                          height="4em",
                          text_align="center",
                          font_size="32px",
                          transform=f"translate(-50%, -50%) rotate({degrees}deg)")
        divs.append(this_div)
html_src = html_tmpl.format(title="circle", content="".join(divs))
show_html(html_src)

Or add random variation:

In [35]:
divs = []
src = "ALPHABET"
for i in range(len(src)):
    ch = src[i]
    center_x = 50 + (30 * math.cos(i * (math.tau / len(src)))) + random.gauss(0, 2)
    center_y = 50 + (30 * math.sin(i * (math.tau / len(src)))) + random.gauss(0, 2)
    n = random.randrange(8, 18)
    for j in range(n):
        degrees = j * (360 / n)
        this_div = mkdiv(src[i],
                          position="absolute",
                          top=f"{center_y}vh",
                          left=f"{center_x}vh",
                          width="1em",
                          height=f"{random.gauss(4, 0.5)}em",
                          text_align="center",
                          font_size=f"{random.gauss(32, 4)}px",
                          transform=f"translate(-50%, -50%) rotate({degrees}deg)")
        divs.append(this_div)
html_src = html_tmpl.format(title="circle", content="".join(divs))
show_html(html_src)

(The [`random.gauss()` function](https://docs.python.org/3/library/random.html#random.gauss) returns numbers from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution).)

## Programming R. L. Draper's "Target Practice"

The final example in this notebook shows how to make a computational version of R. L. Draper's "Target Practice" (reproduced and discussed in S Cearley's ["How to read a concrete poem"](http://www.futureanachronism.com/digitalpaper/HowToReadAConcretePoem.pdf)). This example makes use of some of the same rotation tricks we took advantage of before, but also does some new things: the `<div>` tags are filled with multiple copies of the same word, and prevented from expanding the tag's boundary with the CSS properties `overflow-wrap: anywhere` and `overflow: hidden`. Setting the `z-index` property controls which elements are on "top"; this is important for this example, where the `<div>`s have opaque backgrounds (set with `background: white`). Finally, the `border-radius` property set to `50%` makes these `<div>`s—normally rectangular—appear as circles.

In [36]:
divs = []
words = ["death", "maim", "cripple", "wound"]
n = 4
for i in range(n):
    degrees = random.randrange(360)
    content = words[i] * 1000
    this_div = mkdiv(content,
                      top="50%",
                      left="50%",
                      width=f"{(i+1) * 7}em",
                      height=f"{(i+1) * 7}em",
                      transform=f"translate(-50%, -50%) rotate({degrees}deg)",
                      font_family="Courier",
                      z_index=f"{1000 - i}",     # z-index controls what's in front
                      overflow_wrap="anywhere",  # wrap the text anywhere
                      overflow="hidden",         # hide text that falls outside the box
                      background="white",        # make this opaque
                      border_radius="50%"        # make it circular
                     )
    divs.append(this_div)
html_src = html_tmpl.format(title="centered", content="".join(divs))
show_html(html_src)

## Further ideas and resources

The examples in this notebook are focused on a handful of particular techniques—it's not meant to be exhaustive, but just to give you some code that you can play with and pore over to give you new ideas.

Some quick things that you could play with:

* Try working with *lots* of text.
* Wherever there's a number that is small, make it big. Wherever there's a number that's big, make it small.
* Play with color.
* Play with fonts. (It wouldn't be too difficult to get [Google Fonts](https://fonts.google.com/) working with this code. Hint: Modify the HTML template.)
* Play with [CSS animations](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Animations/Using_CSS_animations). This is a bit trickier, because there isn't an easy way to specify CSS animations in-line. (You'd probably have to modify the HTML template to allow for interpolation of `@keyframe` definitions in the stylesheet, for starters.)

HTML/CSS and print:

* [I totally forgot about print style sheets](https://www.matuzo.at/blog/i-totally-forgot-about-print-style-sheets/) is a good recent overview of CSS styles for producing printed artifacts.
* [Bindery](https://evanbrooks.info/bindery/) is a JavaScript library for doing book layout and design with HTML and CSS.
* [Gutenberg](https://github.com/BafS/Gutenberg): a CSS framework specifically for print

HTML and CSS tutorials:

* [Mozilla Developer Network's HTML/CSS Tutorials](https://developer.mozilla.org/en-US/docs/Web/Tutorials) are highly recommended!
* [Learn Layout](http://learnlayout.com/) is CSS's missing manual. Laconic and instantly useful.
* My HTML/CSS tutorials (perhaps a bit out of date, but deliciously so): [HTML: An Elementary Introduction](http://hypertext.decontextualize.com/elementary/), [CSS with style](http://hypertext.decontextualize.com/css-with-style/), [Your position is clear](http://hypertext.decontextualize.com/your-position-is-clear/).